In [19]:
#====== Ab hier den Code ergänzen ===========

from cryptography.hazmat.primitives import hashes
import sys

# Dieser Command kann auskommentiert werden; dann kann direkt über die Konsole gearbeitet werden
# Beispiel: python BDL_PoW.py wort 2
sys.argv = ["BDL_PoW", "orange", "3"]

InputString = sys.argv[1]  # read first input to the program (prefix)
targetIndex = int(sys.argv[2])  # read second input to the program (target value)

if targetIndex > 4:
    print("In dieser Aufgabe gibt es nur Target-Indizes von 0 bis 4.")
    exit(1)

# Targets (aus dem Forum auf Moodle)
targets = [
    21674354912456743549124567435491245674354912456743549124567435491245674354, 
    3674354912456743549124567435491245674354912456743549124567435491245674354,
    674354912456743549124567435491245674354912456743549124567435491245674354, 
    74354912456743549124567435491245674354912456743549124567435491245674354,
    4354912456743549124567435491245674354912456743549124567435491245674354
]

target = targets[targetIndex]

# ========= Startsuche =========

x = 0  # Start counter

while True:
    # String konstruieren
    NextString = f"{InputString}{x}"
    Data = NextString.encode("utf-8")

    # Hash berechnen (SHA3-256)
    digest = hashes.Hash(hashes.SHA3_256())
    digest.update(Data)
    HashVal = digest.finalize()  # bytes

    # Hash in Integer umwandeln (big-endian)
    HashInt = int.from_bytes(HashVal, byteorder="big",signed=False)

    # Prüfen, ob der Hash kleiner als das Target ist
    if HashInt < target:
        print("=== PoW-Lösung gefunden ===")
        print(f"Target-Index: {targetIndex}")
        print(f"Prefix:       {InputString}")
        print(f"Counter x:    {x}")
        print(f"String:       {NextString}")
        print(f"Hash (hex):   {HashVal.hex()}")
        print(f"Hash (int):   {HashInt}")
        break

    x += 1
#Compute first hash (use SHA3_256 as in Hashing.py)

#Loop (for example while-loop as long as hash value too high):
#  Compute next counter
#  Create new string
#  Compute Hash-Value as Integer of new string

#Print the proof-of-work (PoW)-solution!

=== PoW-Lösung gefunden ===
Target-Index: 3
Prefix:       orange
Counter x:    2635650
String:       orange2635650
Hash (hex):   0000060a4b979e630968c9fe2503858f09616d609383e3c6e21514ce574656be
Hash (int):   41688038341604944567962746751794555372161149104143813825494869078333118


In [3]:
import json
from web3 import Web3

# Connect to local Hardhat node
provider_url = "http://185.48.228.49:8545"
w3 = Web3(Web3.HTTPProvider(provider_url))

if not w3.is_connected():
    raise Exception("❌ Could not connect to Hardhat node at " + provider_url)

print("✅ Connected to Hardhat node")

# Get latest block
latest_block = w3.eth.block_number
print(f"Latest block: {latest_block}")

print("Ordered List of Transactions in the Testnet Ledger")
for i in range(0, latest_block + 1):
    block = w3.eth.get_block(i, full_transactions=True)
    txs = block.transactions
    if txs:
      for tx in txs:
        print("===============================================================")
        print("TX-ID:" + tx.hash.hex())
        print("")
        print("FROM:")
        print(tx["from"])
        print("")
        print("TO:")
        print(tx["to"])
        print("")
        print("AMOUNT (Wei):")
        print(str(tx["value"]))
        print("")

        # Auf Meta-daten greift man wie folgt zu:
        data = tx["input"]

        # Ob Metaden gesetzt sind, kann man z.B. so testen:
        if data != b'':
          try:
            print(data.decode("utf-8"))
          except:
             print("Data could not be decoded to utf-8.")
        #   Do something useful...

        #Decodierung unbedingt mit try-except, falls die Daten nicht utf-8 codiert sind.
          try:
            print(data.decode("utf-8"))
          except:
             print("Data could not be decoded to utf-8.")


✅ Connected to Hardhat node
Latest block: 150
Ordered List of Transactions in the Testnet Ledger
TX-ID:c60d1f972cd93b0ce1053fdd56c23d1594af3a33cf5d7281a7f550b6e036be8c

FROM:
0x0E9E9cff528B0278bFB47886ECf11A9D00A94324

TO:
0x802ba74718b86107F8dF036254506449Ec38f720

AMOUNT (Wei):
100999999960623125000000

TX-ID:4cb62901f5d77a8902f3e21b9b0479bab439150eb751a0b0d09c48ff463cb696

FROM:
0x802ba74718b86107F8dF036254506449Ec38f720

TO:
0x3F59aDA7605BD8763c089a0BB53a1bD2b00494A3

AMOUNT (Wei):
50000000000000000000000

TX-ID:1f6a9efca802fb1ebc9ec2b8507b8ac9c59c5f0af29bab819287c382fbd27d2d

FROM:
0x06d2784af9FCDa2F9B8E1a22688E39408055A4DA

TO:
0x1e4879168E66246B12B7CFC887eFb0404a1aC65E

AMOUNT (Wei):
100999999743244152648129

TX-ID:45b4b5cf3813ca7ffcb426661ff91c8351e9e8fc6d9a00f7dfe515027e221730

FROM:
0xC1338564eA3233F72A8D87bbb1781429c17dbdd8

TO:
0x1e4879168E66246B12B7CFC887eFb0404a1aC65E

AMOUNT (Wei):
100999999745084678204579

TX-ID:1981839d5433abe07fe3177dc1f57185f75460460068832edf073e022a

FROM:
0x43aFCc439071f67E405a6c88a82adC4177Bc3839

TO:
0x802ba74718b86107F8dF036254506449Ec38f720

AMOUNT (Wei):
1000000000000000000

Metadata

1famous2fun
3nut4term
HelloStudents
blockchain
5thing6argue
7space8defy
9place10vast
11denial12coffee

ABCD

In [3]:
import os, sys
from cryptography.hazmat.primitives.ciphers.aead import AESGCMSIV
from cryptography.hazmat.primitives import hashes

# ---- Helper function for your convenience ---------------
def key_from_password(password):
    Data = password.encode()
    digest = hashes.Hash(hashes.SHA3_256())
    digest.update(Data)
    HashVal = digest.finalize() #Data type of result is bytes
    return HashVal

def encrypt_recovery_phrase(key, recoveryPhrase):
    #Encrypt with fixed nonce and no additional data.
    #Note: using a fixed nonce is not good security practice!
    #Technical Comment: For our example, however, this is ok because seed phrases contain sufficient randomness.
    nonce = bytes.fromhex("FFFFFFFFFFFFFFFFFFFFFFFF")
    ad = bytes.fromhex("00")

    #Here we actually encrypt
    data = bytes(recoveryPhrase, 'utf-8') 
    aesgcmsiv = AESGCMSIV(key)
    ciphertext = aesgcmsiv.encrypt(nonce, data, ad)
    return ciphertext


def decrypt_ciphertext(key, ciphertext):
  nonce = bytes.fromhex("FFFFFFFFFFFFFFFFFFFFFFFF")
  ad = bytes.fromhex("00")
  #Here we actually decrypt
  aesgcmsiv = AESGCMSIV(key)
  try: 
    plainData = aesgcmsiv.decrypt(nonce, encryptedData,ad)
  except:
    return "ERROR"
  return plainData.decode("utf-8")
# ------------------------------------------------------

terminated = False

#First read the encrypted file as bytes.
print("Provide Path to Encrypted Recovery File: ")
filePath = "./Encrypted_with_pw.bin"
recoveryPhraseFileEncrypted = open(filePath , "rb")
encryptedData = recoveryPhraseFileEncrypted.read()
recoveryPhraseFileEncrypted.close()

#------ Ask for Password and compute key by hashing password -----
print("Provide the Password that Protects your Seed Phrase")
#Decrypt
pin = 0
while pin < 999999:
    
    key = key_from_password(str(pin))
    res = decrypt_ciphertext(key,encryptedData)
    if res != "ERROR":
      print("Seed Phrase: "+ res )
    pin +=1




Provide Path to Encrypted Recovery File: 
Provide the Password that Protects your Seed Phrase
Seed Phrase: eagle.love.direct.include.avocado.rack.possible.wagon.street.supply.ticket.effort


agle.love.direct.include.avocado.rack.possible.wagon.street.supply.ticket.effort

In [1]:
import json
from web3 import Web3

# Connect to local Hardhat node
provider_url = "http://185.48.228.49:8545"
w3 = Web3(Web3.HTTPProvider(provider_url))

if not w3.is_connected():
    raise Exception("❌ Could not connect to Hardhat node at " + provider_url)

print("✅ Connected to Hardhat node")

# Get latest block
latest_block = w3.eth.block_number
print(f"Latest block: {latest_block}")

print("Ordered List of Transactions in the Testnet Ledger")
for i in range(0, latest_block + 1):
    block = w3.eth.get_block(i, full_transactions=True)
    txs = block.transactions
    if txs:
      for tx in txs:
        print("===============================================================")
        print("TX-ID:" + tx.hash.hex())
        print("")
        print("FROM:")
        print(tx["from"])
        print("")
        print("TO:")
        print(tx["to"])
        print("")
        print("AMOUNT (Wei):")
        print(str(tx["value"]))
        print("")
        data = tx["input"]
        if data != b'':
          print("Raw Data:")
          print(data)
          try:
            text = data.decode("utf-8")
            print("Data Field Decoded (utf-8):" + text)
          except:
            pass


✅ Connected to Hardhat node
Latest block: 165
Ordered List of Transactions in the Testnet Ledger
TX-ID:c60d1f972cd93b0ce1053fdd56c23d1594af3a33cf5d7281a7f550b6e036be8c

FROM:
0x0E9E9cff528B0278bFB47886ECf11A9D00A94324

TO:
0x802ba74718b86107F8dF036254506449Ec38f720

AMOUNT (Wei):
100999999960623125000000

TX-ID:4cb62901f5d77a8902f3e21b9b0479bab439150eb751a0b0d09c48ff463cb696

FROM:
0x802ba74718b86107F8dF036254506449Ec38f720

TO:
0x3F59aDA7605BD8763c089a0BB53a1bD2b00494A3

AMOUNT (Wei):
50000000000000000000000

TX-ID:1f6a9efca802fb1ebc9ec2b8507b8ac9c59c5f0af29bab819287c382fbd27d2d

FROM:
0x06d2784af9FCDa2F9B8E1a22688E39408055A4DA

TO:
0x1e4879168E66246B12B7CFC887eFb0404a1aC65E

AMOUNT (Wei):
100999999743244152648129

TX-ID:45b4b5cf3813ca7ffcb426661ff91c8351e9e8fc6d9a00f7dfe515027e221730

FROM:
0xC1338564eA3233F72A8D87bbb1781429c17dbdd8

TO:
0x1e4879168E66246B12B7CFC887eFb0404a1aC65E

AMOUNT (Wei):
100999999745084678204579

TX-ID:1981839d5433abe07fe3177dc1f57185f75460460068832edf073e022a

In [5]:
from web3 import Web3
import json
import sys

# Connect to local Hardhat node
provider_url = "http://185.48.228.49:8545"
w3 = Web3(Web3.HTTPProvider(provider_url))

if not w3.is_connected():
    raise Exception("❌ Could not connect to Hardhat node at " + provider_url)

print("✅ Connected to Hardhat node")

wallet_addr = "0x9caAF6b5aB6e355F5244F6319EA6b3BB83146f21"
address = Web3.to_checksum_address(wallet_addr)  # Use the correct method for checksumm

# Get balance in Wei
balance_wei = w3.eth.get_balance(address)

# Convert to Ether
balance_eth = w3.from_wei(balance_wei, "ether")

print(f"Balance of {address}: {balance_eth} ETH")


✅ Connected to Hardhat node
Balance of 0x9caAF6b5aB6e355F5244F6319EA6b3BB83146f21: 2401.893036537412294 ETH


In [6]:
from web3 import Web3
import json
import sys

# We read address and private key from a json file for simplicity. NOTE THAT THIS IS NOT SAFE.
# The file is not encrypted (e.g. using a password to decrypt). We do this because the wallets we use in the lecture are for our testnet and not intended to hold real ETH on Mainnet.
# Notice that software wallets (like Metamask) protect such sensitive files with a password-based encryption method 
# and only decrypt when needed (and after asking for your password).
def load_wallet_config():  
    """Load configuration from a JSON file named after the player."""
    with open("wallet.json", 'r') as file:
        config = json.load(file)
        return config['wallet_addr'], config['private_key']

# Connect to local Hardhat node
provider_url = "http://185.48.228.49:8545"
w3 = Web3(Web3.HTTPProvider(provider_url))

if not w3.is_connected():
    raise Exception("❌ Could not connect to Hardhat node at " + provider_url)

print("✅ Connected to Hardhat node")


# Load Key

#If you don't have a key yet or don't want to export from Metamask, you can generate a new one with Python using the following instruction (which you should run separately or directly in the command line).
# Don't forget to send coins on our Testnet from your existing account to the new one before using it here.

# acc = w3.eth.account.create()
# print(f'private key={w3.to_hex(acc.key)}, account={acc.address}')

wallet_addr, private_key = load_wallet_config()
address = Web3.to_checksum_address(wallet_addr)  # Use the correct method for checksumm

# --- Arg1: Recipient ---
receiver_address = "0xabaA2797730a271d1FE8c9b46a4c49b41E93C655"

# --- Arg2: Amount (ETH) ---
amount = int(1)

# Die Nonce zählt pro Adresse, wieviele Txs schon verschickt wurden damit. Mehr dazu in der Self-Study Woche 3.
nonce = w3.eth.get_transaction_count(address)
tx = {
    "nonce": nonce,
    "to": receiver_address,
    "value": w3.to_wei(amount, "ether"),
    "gas": 30000,  # add a gas limit that suffices for sure.
    "gasPrice": w3.eth.gas_price,
    "chainId": 24888  # our testnet
    }

# --- Arg3: Metadata (if present) ---
if len(sys.argv) > 3:
    metadata = sys.argv[3]
    metadata_hex = metadata.encode("utf-8").hex()  # convert to hex
    metadata_field = "0x" + metadata_hex
    tx["data"] = metadata_field

# --- Sign transaction ---
signed_tx = w3.eth.account.sign_transaction(tx, private_key)

# --- Send transaction ---
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

print("Transaction sent with hash:", tx_hash.hex())


✅ Connected to Hardhat node
Transaction sent with hash: 2df07495792fe0b41a8229c9f066211f4fad14b78a61718ab12ce4b1f800cc31
